## Import

In [1]:
import pandas as pd
import sqlite3
import os

In [ ]:
current_dir = os.getcwd()
big_dir = os.path.dirname(current_dir)
path_db = os.path.join(big_dir, 'data', 'interim', 'olist_database.db')
conn = sqlite3.connect(path_db)
print(f"Connected to database at: {path_db}")

main_query = """
WITH
items_agg AS (
    SELECT
        order_id,
        SUM(price) AS total_product_price,
        SUM(freight_value) AS total_freight_value
    FROM olist_order_items_dataset
    GROUP BY order_id
),

payments_agg AS (
    SELECT
        order_id,
        SUM(payment_value) AS total_payment_value,
        GROUP_CONCAT(payment_type, ', ') AS payment_types
    FROM olist_order_payments_dataset
    GROUP BY order_id
)

SELECT
    -- Ключі
    o.order_id,
    o.customer_id,

    -- Дати (для розрахунків)
    o.order_purchase_timestamp,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,

    -- Клієнт
    c.customer_city,
    c.customer_state,

    -- Відгук (наша ціль)
    r.review_score,

    -- Гроші (з наших WITH-таблиць)
    i.total_product_price,
    i.total_freight_value,
    p.total_payment_value,
    p.payment_types
    
FROM
    olist_orders_dataset AS o
LEFT JOIN
    olist_order_reviews_dataset AS r ON o.order_id = r.order_id
LEFT JOIN
    olist_customers_dataset AS c ON o.customer_id = c.customer_id
LEFT JOIN
    items_agg AS i ON o.order_id = i.order_id  
LEFT JOIN
    payments_agg AS p ON o.order_id = p.order_id 

WHERE
    o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
    AND o.order_purchase_timestamp IS NOT NULL
    AND o.order_estimated_delivery_date IS NOT NULL
    AND i.total_product_price IS NOT NULL
    AND i.total_product_price > 0
;
"""


print("Executing main SQL query... Це може зайняти хвилину...")
df_main = pd.read_sql_query(main_query, conn)

conn.close()
print(f"Successfully loaded and transformed data! Created DataFrame with {len(df_main)} rows.")

df_main.head()

Connected to database at: d:\my_projects\project_Brazilian_ECommerce_by_Olist\data\interim\olist_database.db
Executing main SQL query... Це може зайняти хвилину...
Successfully loaded and transformed data! Created DataFrame with 96999 rows.


,order_id,customer_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,customer_city,customer_state,review_score,total_product_price,total_freight_value,total_payment_value,payment_types
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,sao paulo,SP,4.0,29.99,8.72,38.71,"credit_card, voucher, voucher"
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,barreiras,BA,4.0,118.70,22.76,141.46,boleto
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,vianopolis,GO,5.0,159.90,19.22,179.12,credit_card
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,sao goncalo do amarante,RN,5.0,45.00,27.20,72.20,credit_card
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,santo andre,SP,5.0,19.90,8.72,28.62,credit_card


## 1. SQL Query Analysis

This query is designed to extract all necessary raw data from 8 different tables and combine them into a single, clean table for analysis.
Here are the key logical decisions:
1. `WITH` Clauses (CTEs): Before the main query, two temporary "mini-tables" (`items_agg` and `payments_agg`) were created using Common Table Expressions.
* Why? The `order_items` and `order_payments` tables have a one-to-many relationship with `orders` (one order can have many items). A simple `JOIN` would cause massive data duplication (a "fan-out"), making all `SUM()` calculations incorrect.
* Solution: We first pre-aggregate (using `SUM()` and `GROUP BY`) these tables to get a single row per order_id, which we can then safely join.

2. `LEFT JOIN`: The main query uses `LEFT JOIN` instead of a standard `INNER JOIN`.
* Why? Our goal is to analyze all delivered orders from the main `olist_orders_dataset table`.
Problem: If we used an `INNER JOIN`, any order that was missing a review (or a customer record) would be dropped from our analysis. This would skew our results.
* Solution: `LEFT JOIN` ensures that we keep all rows from the "left" table (`olist_orders_dataset`) and simply fill in `NULL` values for any missing data from the "right" tables (like `review_score`).

3. `WHERE` Clause: The query filters for `order_status = 'delivered'` and checks for `IS NOT NULL`.
* Why? We can only analyze customer satisfaction on orders that were actually completed. Filtering out canceled or shipped orders, as well as rows with missing data, ensures our final dataset is clean and relevant to the business problem.